In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import api
import matplotlib.pyplot as plt
from datetime import date

# Enter your Toggl email and password here.
TOGGL_EMAIL = r''
TOGGL_PASSWORD = r''

_api = api.TogglApi(TOGGL_EMAIL, TOGGL_PASSWORD)

In [ ]:
# Call the `me` endpoint and get the default workspace ID.
my_data = _api.get_my_data()
WORKSPACE_ID = my_data['default_workspace_id']

In [ ]:
# Load project data.
projects = _api.get_project_data(WORKSPACE_ID)

In [ ]:
# Calculate the sum of hours tracked.
print(f"You tracked a total of {sum(p.actual_hours for p in projects)} hours.")

# Sort projects by time spent.
projects.sort(key=lambda p: p.actual_hours, reverse=True)
print('\n'.join(f"{p.name}: {p.actual_hours} hours" for p in projects))

# Create a mapping of project ID to name, which we'll need later.
# TODO: Better name
project_names = {p.id: p.name for p in projects}

In [ ]:
# Make a simple bar plot of projects by time spent.
# Note: adding a semicolon ";" at the end of the plot statement hides the annoying text that the notebook would normally print.
fig, ax = plt.subplots()
fig.suptitle('Time Spent (Bar)')
ax.barh(
    [p.name for p in projects],
    [p.actual_hours for p in projects],
);

In [ ]:
# Make a pie plot of the top 10 projects.
n = min(10, len(projects))

fig, ax = plt.subplots()
fig.suptitle(f"Time Spent, Top {n}")
ax.pie(
    [p.actual_hours for p in projects[:n]],
    labels=[p.name for p in projects[:n]],
    autopct="%1.1f%%",
);

In [ ]:
# Load all tracked time entries from `START_DATE` until `END_DATE`, inclusive.
START_DATE = date(2023, 9, 1)
END_DATE = date(2023, 10, 1)
time_entries = _api.get_time_entries(START_DATE, END_DATE)

print(f"Loaded {len(time_entries)} time entries.")

# Generate a Markdown table with some basic statistics.
table = "| Project | Num Entires | Time Spent | Avg. Time |\n"
table += "| --- | --- | --- |\n"
import statistics
for project_id, pname in project_names.items():
    project_entries = list(filter(lambda t: (t.project_id == project_id), time_entries))
    table += f"| {pname} | {len(project_entries)} | {sum(t.duration for t in project_entries)} | {statistics.mean(t.duration for t in project_entries)} |\n"

from IPython.display import display, Markdown, Latex
display(Markdown(table))


In [ ]:
print(project_names)